In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkStreamTrigger")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/05/24 21:17:57 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/24 21:17:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/24 21:17:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# pre-requistes

# we need a server running on port 9999, that provides text stream to spark
# spark stream is a client, that will connect ot a server running on port 9999

# terminal, 

# nc -lk 9999

In [4]:
# Structured Streaming 
# based on Data Frame/ internally there is RDD
# Micro Batch
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, represent data comes from TCP/IP socket
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark shall connect the server running on port 9999
# then it will wait the input from server
# user/developer can write something on server console and Press Enter key
# to send to spark / client

22/05/24 21:19:34 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
lineStreamDf.printSchema()
# show will not work for stream, it will cause error
#lineStreamDf.show() # good for batchs, not for stream

root
 |-- value: string (nullable = true)



In [6]:
# print data on console 
# for stream write, we should use writeStream
# for batch use write
# append is mode
# console is keyword, present console output
# writeStream is an action, which runs the whole show,
# reading data, trigger window etc by writeStream
# in general 1 sec trigger used if not mentioned, default

# trigger time 10 second processingTime
# 10 second micro batch interval
# echoOnconsole is query
echoOnconsole = lineStreamDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .trigger(processingTime='10 seconds')\
                .start()

# start basically start the streaming, listen for input
# as input arrives, it starts processing
# listen for jupyter console for the output

22/05/24 21:20:41 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a89f7b5d-c77e-4acd-a751-3e35a94cc325. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
|    8|
|    9|
|   10|
+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|value|
+-----+
|  one|
|  two|
|three|
+-----+



In [7]:
# query id
echoOnconsole.id

'b5e408e7-64af-48c8-9b07-745ff4847dba'

In [8]:
# auto generated or user given
echoOnconsole.name 

In [9]:
# if any exception
echoOnconsole.exception

<bound method StreamingQuery.exception of <pyspark.sql.streaming.StreamingQuery object at 0x7f2c284aa6d0>>

In [10]:
echoOnconsole.recentProgress

[{'id': 'b5e408e7-64af-48c8-9b07-745ff4847dba',
  'runId': '7960d41f-f48b-483f-86a0-02b115cd3492',
  'name': None,
  'timestamp': '2022-05-24T15:50:42.146Z',
  'batchId': 0,
  'numInputRows': 0,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 0.0,
  'durationMs': {'addBatch': 6316,
   'getBatch': 15,
   'latestOffset': 0,
   'queryPlanning': 1284,
   'triggerExecution': 8072,
   'walCommit': 241},
  'stateOperators': [],
  'sources': [{'description': 'TextSocketV2[host: localhost, port: 9999]',
    'startOffset': None,
    'endOffset': -1,
    'numInputRows': 0,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 0.0}],
  'sink': {'description': 'org.apache.spark.sql.execution.streaming.ConsoleTable$@3add5d23',
   'numOutputRows': 0}},
 {'id': 'b5e408e7-64af-48c8-9b07-745ff4847dba',
  'runId': '7960d41f-f48b-483f-86a0-02b115cd3492',
  'name': None,
  'timestamp': '2022-05-24T15:51:10.001Z',
  'batchId': 1,
  'numInputRows': 0,
  'inputRowsPerSecond': 0.0,
  'processed

In [ ]:
# await for termination 
# block next operations until any termination
echoOnconsole.awaitTermination()
print("await after") # this statement won't be executed until stream close
# now you have to go to server, stop the server using Ctrl + C